# **EDA King County Housing Data**

**Stakeholder:** Thomas Hansen.

**Thomas situation:** Thomas wants to buy a house. He has 5 children, no money and would like to know where suitable houses are located and when it is cheap to buy.

Main Questions:

1. Where??
2. When??

Requirements:
1. Neighborhood is not a ghetto
2. Neighborhood is lively (no village)
3. House has enough space for 1 adult and 5 children
4. House is cheap




## Import libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from plotly.subplots import make_subplots
import altair as alt
import numpy as np
import statsmodels.api as sms

## Import dataframe 


* get an overview

In [ ]:
house = pd.read_csv('../data/King_County_House_prices_dataset.csv')
house.describe()

## Create new variables

1. Price per Squarefoot (price_sqft)
2. Month when house was sold (month_sold)

In [ ]:
# price per sqft
house['price_sqft'] = house.price / house.sqft_living
new_house = house.copy()


# month_sold
new_house['date']=pd.to_datetime(new_house['date'],format='%m/%d/%Y')
new_house['month_sold'] = new_house['date'].dt.month

## Filter dataframe to get relevant data

**Assumptions:**

* Where many houses are sold, there tend to be more houses -> **lively neighborhood** - neighborhoods are taken that are above the median of the sum of sold houses per zip code.
* Quiet uncrowded neighborhoods are more likely to be in the countryside and Houses have large lots -> If the 25% quartile of the land lots within the zip code is greater than the median of the land lots of all areas the zipcode is going to be filtered out.
* The worst neighborhoods have the lowest price per square foot -> zip codes within the 75% quartile is lower than the 25% quartile of all neighborhoods are going to be filtered out.
* Thomas needs at least 4 bedrooms and 1184 square feet of living space




In [ ]:
# to see the data --> uncomment the codelines!
# median sqft land lot
# new_house.sqft_lot15.describe()

# take median zipcodes = 282 houses per zipcode
# new_house.groupby('zipcode').count().id.describe()

## Filter Dataframe

* to filter the data I have created lists with zipcodes

In [ ]:
# median land lot sqft 7620 

big = new_house.groupby('zipcode').sqft_lot15.describe()['25%'].reset_index()
big = big[big['25%'] > 7620]
big_list = big.zipcode.to_list()

# Q25 price per sqft = 177

ghetto = new_house.groupby('zipcode').price_sqft.describe()["75%"].reset_index()
ghetto = ghetto[ghetto['75%'] < 177]
ghetto_list = ghetto.zipcode.to_list()

# filter the zipcodes with more sold houses (bigger than median) and add them to the list zips

top_zip = new_house.groupby('zipcode').count().query('id > 282').reset_index()
zips = top_zip.zipcode.to_list()

# filter dataframe
new_house = new_house.query('zipcode in @zips and zipcode not in @ ghetto_list and zipcode not in @big_list and sqft_living > 1184 and bedrooms >= 4 ')


* for a better overview I have created the new dataframe **final**, which contains only the variables that are of interest
* in addition I have created a new dataframe in which longitudinal and lateral geodata is added to each zipcode, as well as the median square foot price in this zipcode
* the name of this dataframe is **final_frame**
* i build **final_frame** for better geo plotting

In [ ]:
# new dataframe with relevant variables

final = new_house[['price', 'bedrooms', 'sqft_living', 'zipcode', 'month_sold', 'lat', 'long', 'price_sqft']].copy()

# new dataframe final_frame

final_long = final.groupby('zipcode').describe().long['mean'].reset_index()
final_long.rename(columns={'mean': 'long'}, inplace=True)
final_lat = final.groupby('zipcode').describe().lat['mean'].reset_index()
final_lat.rename(columns={'mean': 'lat'}, inplace=True)
final_zips = pd.merge(final_lat, final_long, on='zipcode')
final_price = final.groupby('zipcode').describe().price_sqft['50%'].reset_index()
final_price.rename(columns={'50%': 'price_sqft'}, inplace=True)
final_frame = pd.merge(final_zips, final_price, on='zipcode')


## Create plots to show Results

### 1. Plot 

* shows all possible neighborhoods after filtering the data on the map

In [ ]:
fig = px.scatter_mapbox(final_frame, lat=final_frame.lat, lon=final_frame.long, width=800, height=600, color="price_sqft")
# change default for continuous color scale
px.defaults.color_continuous_scale = px.colors.sequential.Agsunset
fig.update_geos(fitbounds='locations')
fig.update_layout(title='Location - "Nice neighborhood"')

fig.update_traces(marker=dict(size=15))
fig.update_layout(mapbox_style="open-street-map")
fig.show()


### 2. Plot 

* shows the top three neighborhoods with regard to the price per square foot 


In [ ]:
top3_zip = [98042, 98038, 98058]
top = final_frame.query('zipcode in @top3_zip')

fig = px.scatter_mapbox(top, lat=top.lat, lon=top.long, width=800, height=600)
 
fig.update_geos(fitbounds='locations')
fig.update_layout(mapbox_style="open-street-map", title="Location of the three best neighborhoods")
fig.update_traces(marker=dict(size=30))
fig.show()

## 3. Plot

* comparison of average price per square foot bewteen **all neighborhoods** with the **nice neighborhoods** as well as the **top three neighborhoods (nice%cheap)**

In [ ]:
nice = final.query('zipcode in @top3_zip')
house_list = house.price_sqft.to_list()
new_house_list = final.price_sqft.to_list()
top_list = nice.price_sqft.to_list()
dic = {'all neighborhoods': house_list, 'nice neighborhoods': new_house_list, 'nice & cheap NH': top_list}


fig, ax = plt.subplots()
ax.boxplot(dic.values())
ax.set_xticklabels(dic.keys())
ax.set_ylabel('Price per square foot [$]', fontsize=15, fontweight='bold')
ax.set_title('Price per square foot', fontsize=18, fontweight='bold')
ax.set_xticklabels(dic.keys(), fontsize=12, fontweight='bold')
fig.set_size_inches(10, 8, forward=True)
plt.savefig('Price per square foot')

* range of prices per square foot become lower as filter increases

## 4. Plot - Timing?!

* The graph is intended to show the changes in price per square foot within the top 3 neighborhoods over the months
* In addition, the number of home sales per month should be presented too


In [ ]:
# building two lists for plotting "Sales & sqft price per month

sells = nice.groupby('month_sold').count().reset_index()
sells_list = sells.price.to_list() # list list of all house sales per month

sqft = nice.groupby('month_sold').describe().price_sqft.reset_index()
sqft_list = sqft['mean'].to_list() # list of mean price per sqft per month


In [ ]:
x = month = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Okt','Nov','Dez']
y = sqft_list
y1 = sells_list

fig, ax1 = plt.subplots() # initializes figure and plots

ax2 = ax1.twinx() # applies twinx to ax2, which is the second y axis. 

sns.lineplot(x = x, y = y, ax = ax1, marker = 'o', color = 'blue') # plots the first set of data, and sets it to ax1. 
sns.lineplot(x = x, y = y1, marker = 'o', color = 'red', ax = ax2) # plots the second set, and sets to ax2. 

# these lines add the annotations for the plot. 

ax1.set_ylabel('Average price per square foot [$]', color='b',fontsize=13)
ax2.set_ylabel('Number of house sales', color='r', fontsize=13)
ax1.set_title('Sales & Sqft Price per month', fontsize=18, fontweight='bold')
ax1.set_xticklabels(x, fontsize=12)

fig.set_size_inches(13, 5, forward=True)
plt.show()
#plt.savefig('Sales & Sqft Price per month')

* the price seems to behave typically according to the supply and demand principle -> the more houses are sold per month, the cheaper the square foot price becomes
* according to these data it is not recommended to buy at the beginning of the year
* the second half of the year seems to be the best time to buy

### Let's take a look to the next graph! 

In [ ]:

sns.lmplot(x='month_sold', y='price_sqft', data=nice, x_estimator=np.mean)

* the regression line showed a slight downward trend with regard to the price per square meter over the year, but no significance can be determined here on the basis of a regression analysis.

In [ ]:
# Prepare data for modeling
# Choose the predictor and add a constant term
#X = top.month_sold
#y = top.price_sqft_x
#X = sms.add_constant(X)

# Create an OLS model
#model = sms.OLS(y, X)
# use the data to calculate the intercept and slope
#results = model.fit()
# return the output of the model
#results.summary()

## Have a quick look on other variables

let's see how good **condition** and **grade** is in the recommendet top 3 neighborhoods in comparison to the whole dataset

In [ ]:
# top 3
gesamt = house.query('zipcode in @top3_zip')
gesamt.describe()

In [ ]:
# whole dataset
house.describe()

## Summery 

**Recommended neighborhoods/ towns for a house purchase:**

* Renton
    Average price per square foot = 160.59 $

* Covington
    Average price per square foot = 143.95 $

* Maple Valley
    Average price per square foot = 160.17 $

**Recommended timing for a house purchase:**

* After the first quarter of the year.



## Further steps

There is a lot more i can do. Maybe i will think about it in a later but I'm done :D